# ROI Analysis Example

This notebook demonstrates how to perform return on investment analysis for health interventions.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

# Add the scripts directory to the path
sys.path.append(os.path.join(os.pardir, 'scripts'))
sys.path.append(os.path.join(os.pardir, 'scripts', 'models'))
sys.path.append(os.path.join(os.pardir, 'scripts', 'core'))

In [ ]:
# Import the ROI model
# Note: We don't have a specific ROI engine, but ROI can be calculated from other analysis results
# For this example, we'll create a simple ROI calculation based on cost-effectiveness data

# Define a simple ROI calculation function
def calculate_roi(cost, effect, wtp, time_horizon=5):
    """
    Calculate ROI based on cost-effectiveness data.
    
    Args:
        cost: Total cost of intervention
        effect: Total health effect (e.g., QALYs gained)
        wtp: Willingness-to-pay threshold
        time_horizon: Time horizon for analysis
        
    Returns:
        ROI as a percentage
    """
    benefit = effect * wtp  # Convert health effects to monetary value
    net_benefit = benefit - cost
    roi = (net_benefit / cost) * 100
    return roi

In [ ]:
# Define parameters
strategies = ['ECT', 'IV-KA', 'PO-KA']
costs = [5000, 7500, 6000]  # Example costs
effects = [0.6, 0.8, 0.7]   # Example QALYs gained
wtp_threshold = 50000       # Willingness-to-pay per QALY

# Calculate ROI for each strategy
roi_results = []
for i, strategy in enumerate(strategies):
    cost = costs[i]
    effect = effects[i]
    roi = calculate_roi(cost, effect, wtp_threshold)
    
    roi_results.append({
        'strategy': strategy,
        'cost': cost,
        'effect': effect,
        'benefit': effect * wtp_threshold,  # Monetary value of health benefits
        'net_benefit': effect * wtp_threshold - cost,  # Net monetary benefit
        'roi': roi
    })

roi_df = pd.DataFrame(roi_results)
print(roi_df)

In [ ]:
# Visualize ROI results
fig, ax = plt.subplots(2, 2, figsize=(15, 10))

# ROI by strategy
ax[0, 0].bar(roi_df['strategy'], roi_df['roi'], color=['blue', 'green', 'orange'])
ax[0, 0].set_title('Return on Investment by Strategy')
ax[0, 0].set_ylabel('ROI (%)')
ax[0, 0].grid(True, alpha=0.3)

# Costs vs Effects scatter plot
scatter = ax[0, 1].scatter(roi_df['effect'], roi_df['cost'], 
                         s=roi_df['roi']*10, c=roi_df['roi'], cmap='viridis')
for i, strategy in enumerate(roi_df['strategy']):
    ax[0, 1].annotate(strategy, (roi_df['effect'].iloc[i], roi_df['cost'].iloc[i]))
ax[0, 1].set_xlabel('Health Effects (QALYs)')
ax[0, 1].set_ylabel('Costs ($AUD)')
ax[0, 1].set_title('Cost-Effectiveness Plane with ROI as Bubble Size')
plt.colorbar(scatter, ax=ax[0, 1], label='ROI (%)')

# Net benefit bar chart
ax[1, 0].bar(roi_df['strategy'], roi_df['net_benefit'], color=['blue', 'green', 'orange'])
ax[1, 0].set_title('Net Monetary Benefit by Strategy')
ax[1, 0].set_ylabel('Net Monetary Benefit ($AUD)')
ax[1, 0].grid(True, alpha=0.3)

# Cost breakdown
cost_breakdown = roi_df.set_index('strategy')[['cost', 'benefit']].rename(columns={'cost': 'Cost', 'benefit': 'Value of Health Gains'})
cost_breakdown.plot(kind='bar', ax=ax[1, 1], stacked=False)
ax[1, 1].set_title('Cost vs Value of Health Gains')
ax[1, 1].set_ylabel('Amount ($AUD)')
ax[1, 1].legend()
ax[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Calculate ROI sensitivity to WTP threshold
wtp_range = np.arange(10000, 100001, 5000)
sensitivity_data = []

for wtp in wtp_range:
    for i, strategy in enumerate(strategies):
        cost = costs[i]
        effect = effects[i]
        roi = calculate_roi(cost, effect, wtp)
        
        sensitivity_data.append({
            'strategy': strategy,
            'wtp': wtp,
            'roi': roi
        })

sensitivity_df = pd.DataFrame(sensitivity_data)

# Plot sensitivity
plt.figure(figsize=(10, 6))
for strategy in strategies:
    strategy_data = sensitivity_df[sensitivity_df['strategy'] == strategy]
    plt.plot(strategy_data['wtp'], strategy_data['roi'], label=strategy, linewidth=2)

plt.axhline(y=0, color='red', linestyle='--', label='Break-even (0% ROI)')
plt.xlabel('Willingness-to-Pay Threshold ($/QALY)')
plt.ylabel('ROI (%)')
plt.title('ROI Sensitivity to Willingness-to-Pay Threshold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## Next Steps

1. Integrate with PSA data for probabilistic ROI analysis
2. Consider time value of money with net present value calculations
3. Include implementation costs and timelines
4. Compare ROI across different health interventions